### В данном скрипте из эксель-файла (который был получен в результате парсинга минфина) выбираются значения среднемесячной заработной платы по регионам и объединяются с train и test датасетами. Так как названия регионов в датасетах и эксель-файле отличаются, то названия в эксель-файле были переименованы в соответстсвии с train и test.

### В результате получены датасета train и test, куда добавлены фичи плотность населения, заработная плата.

In [1]:
import pandas as pd
import numpy as np

In [2]:
xls = pd.ExcelFile('tab4-zpl.xlsx')
zp = pd.read_excel(xls, sheet_name='с 2018')
zp

,"Среднемесячная номинальная начисленная заработная плата работников по полному кругу организаций в целом по экономике по субъектам Российской Федерации с 2018 года , рублей",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,2018.0,2019.0,2020.0,2021.0
1,Российская Федерация,43724.0,47867.0,51344.0,57244.0
2,Центральный федеральный округ,54689.0,60771.0,65319.0,73548.0
3,Белгородская область,31852.0,34615.0,37442.0,41775.0
4,Брянская область,27251.0,29853.0,31946.0,35582.0
...,...,...,...,...,...
93,Амурская область,42315.0,47234.0,52430.0,59098.0
94,Магаданская область,85631.0,94856.0,102843.0,112852.0
95,Сахалинская область,77499.0,87418.0,92518.0,94285.0
96,Еврейская авт.область,39242.0,42400.0,46237.0,50075.0


In [3]:
zp.rename({'Среднемесячная номинальная начисленная заработная плата работников по полному кругу организаций в целом по экономике по субъектам Российской Федерации с 2018 года , рублей':
          'subject_name', 'Unnamed: 3':'salary'}, axis=1, inplace=True)
zp.head()

,subject_name,Unnamed: 1,Unnamed: 2,salary,Unnamed: 4
0,NaN,2018.0,2019.0,2020.0,2021.0
1,Российская Федерация,43724.0,47867.0,51344.0,57244.0
2,Центральный федеральный округ,54689.0,60771.0,65319.0,73548.0
3,Белгородская область,31852.0,34615.0,37442.0,41775.0
4,Брянская область,27251.0,29853.0,31946.0,35582.0


In [4]:
zp.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 4'], axis=1, inplace=True)
zp = zp.iloc[3:, :]
zp.head()

,subject_name,salary
3,Белгородская область,37442.0
4,Брянская область,31946.0
5,Владимирская область,35240.0
6,Воронежская область,36317.0
7,Ивановская область,29083.0


In [5]:
zp = zp.reset_index()
zp.drop('index', axis=1, inplace=True)

In [6]:
zp.head()

,subject_name,salary
0,Белгородская область,37442.0
1,Брянская область,31946.0
2,Владимирская область,35240.0
3,Воронежская область,36317.0
4,Ивановская область,29083.0


In [7]:
zp['subject_name'] = zp['subject_name'].str.replace(' край', '')
zp['subject_name'] = zp['subject_name'].str.replace(' область', '')
zp['subject_name'] = zp['subject_name'].str.replace('Республика ', '')
zp['subject_name'] = zp['subject_name'].str.replace(' Республика', '')
zp['subject_name'] = zp['subject_name'].str.replace('г. ', '')
zp['subject_name'] = zp['subject_name'].str.replace('г.', '')
zp['subject_name'] = zp['subject_name'].str.replace(' авт.округ', '')
zp['subject_name'] = zp['subject_name'].str.replace(' авт.область', '')
zp['subject_name'] = zp['subject_name'].str.replace('(', '')
zp['subject_name'] = zp['subject_name'].str.replace(')', '')

In [11]:
zp['subject_name'] = zp['subject_name'].str.replace('Арханльская', 'Архангельская')
zp['subject_name'] = zp['subject_name'].str.replace('Белродская', 'Белгородская')
zp['subject_name'] = zp['subject_name'].str.replace('Володская', 'Вологодская')
zp['subject_name'] = zp['subject_name'].str.replace('Калининадская', 'Калининградская')
zp['subject_name'] = zp['subject_name'].str.replace('Ленинадская', 'Ленинградская')
zp['subject_name'] = zp['subject_name'].str.replace('Новродская', 'Новгородская')
zp['subject_name'] = zp['subject_name'].str.replace('Адыя', 'Адыгея')
zp['subject_name'] = zp['subject_name'].str.replace('Воладская', 'Волгоградская')
zp['subject_name'] = zp['subject_name'].str.replace('Дастан', 'Дагестан')
zp['subject_name'] = zp['subject_name'].str.replace('Иншетия', 'Ингушетия')
zp['subject_name'] = zp['subject_name'].str.replace('Оренбуркая', 'Оренбургская')
zp['subject_name'] = zp['subject_name'].str.replace('Курнская', 'Курганская')
zp['subject_name'] = zp['subject_name'].str.replace('Ханты-Мансийский авт.окру- Юа',
                                                    'Ханты-Мансийский Автономный Округ - Югра')
zp['subject_name'] = zp['subject_name'].str.replace('Саха Якутия',
                                                    'Саха /Якутия/')
zp['subject_name'] = zp['subject_name'].str.replace('Маданская',
                                                    'Магаданская')

In [12]:
zp['subject_name'].unique()

array(['Белгородская  ', 'Брянская', 'Владимирская', 'Воронежская',
       'Ивановская', 'Калужская', 'Костромская ', 'Курская', 'Липецкая',
       'Московская', 'Орловская', 'Рязанская', 'Смоленская',
       'Тамбовская ', 'Тверская', 'Тульская', 'Ярославская   ', 'Москва',
       'Северо-Западный федеральный окру', 'Карелия', 'Коми',
       'Архангельская', 'в том числе Ненецкий',
       'Архангельская без авт. окру', 'Вологодская', 'Калининградская',
       'Ленинградская', 'Мурманская', 'Новгородская', 'Псковская',
       'Санкт-Петербург', 'Южный федеральный окру', 'Адыгея', 'Калмыкия',
       'Крым', 'Краснодарский', 'Астраханская', 'Волгоградская',
       'Ростовская', 'Севастополь', 'Северо-Кавказский федеральный округ',
       'Дагестан', 'Ингушетия', 'Кабардино-Балкарская',
       'Карачаево-Черкесская', 'Северная Осетия - Алания', 'Чеченская',
       'Ставропольский', 'Приволжский федеральный округ', 'Башкортостан',
       'Марий Эл', 'Мордовия', 'Татарстан', 'Удмуртская', '

In [13]:
train = pd.read_csv('train_v3.csv')
train

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f25,f26,f27,f28,f29,f30,percent_zeros,percent_ones,population,area
0,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00101,0.00103,...,NaN,NaN,NaN,NaN,0.01737,0.0,0.975096,0.024904,12635466,2561
1,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00315,0.00322,...,0.66667,0.00855,0.01709,0.01709,0.00784,0.0,0.975096,0.024904,12635466,2561
2,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00173,0.00175,...,0.63569,0.03718,0.03346,0.00744,0.01316,0.0,0.975096,0.024904,12635466,2561
3,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00000,0.00000,...,NaN,NaN,NaN,NaN,0.02703,0.0,0.975096,0.024904,12635466,2561
4,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00035,0.00033,...,NaN,NaN,NaN,NaN,0.01737,0.0,0.975096,0.024904,12635466,2561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294253,0,2020-12-01,Область,Магаданская,Магадан,8617155b7ffffff,59.647301,150.880967,0.00001,0.00001,...,0.82927,0.02439,0.00000,0.02439,0.03226,0.0,1.000000,0.000000,137767,462464
294254,0,2020-12-01,Область,Магаданская,Магадан,86171424fffffff,59.749702,150.900795,NaN,NaN,...,0.86301,0.00000,0.00000,0.02740,0.03774,0.0,1.000000,0.000000,137767,462464
294255,0,2020-12-01,Область,Магаданская,Магадан,861715c87ffffff,59.906078,150.869685,0.00076,0.00086,...,NaN,NaN,NaN,NaN,0.08696,0.0,1.000000,0.000000,137767,462464
294256,0,2020-12-01,Автономный Округ,Чукотский,Певек,8604ebc5fffffff,69.703205,170.185894,0.00001,0.00002,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,1.000000,0.000000,50040,721481


In [14]:
train_zp_merged = train.merge(zp, how='left', on='subject_name')
train_zp_merged

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f26,f27,f28,f29,f30,percent_zeros,percent_ones,population,area,salary
0,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00101,0.00103,...,NaN,NaN,NaN,0.01737,0.0,0.975096,0.024904,12635466,2561,100070.0
1,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00315,0.00322,...,0.00855,0.01709,0.01709,0.00784,0.0,0.975096,0.024904,12635466,2561,100070.0
2,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00173,0.00175,...,0.03718,0.03346,0.00744,0.01316,0.0,0.975096,0.024904,12635466,2561,100070.0
3,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00000,0.00000,...,NaN,NaN,NaN,0.02703,0.0,0.975096,0.024904,12635466,2561,100070.0
4,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00035,0.00033,...,NaN,NaN,NaN,0.01737,0.0,0.975096,0.024904,12635466,2561,100070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294253,0,2020-12-01,Область,Магаданская,Магадан,8617155b7ffffff,59.647301,150.880967,0.00001,0.00001,...,0.02439,0.00000,0.02439,0.03226,0.0,1.000000,0.000000,137767,462464,102843.0
294254,0,2020-12-01,Область,Магаданская,Магадан,86171424fffffff,59.749702,150.900795,NaN,NaN,...,0.00000,0.00000,0.02740,0.03774,0.0,1.000000,0.000000,137767,462464,102843.0
294255,0,2020-12-01,Область,Магаданская,Магадан,861715c87ffffff,59.906078,150.869685,0.00076,0.00086,...,NaN,NaN,NaN,0.08696,0.0,1.000000,0.000000,137767,462464,102843.0
294256,0,2020-12-01,Автономный Округ,Чукотский,Певек,8604ebc5fffffff,69.703205,170.185894,0.00001,0.00002,...,0.00000,0.00000,0.00000,0.00000,0.0,1.000000,0.000000,50040,721481,120641.0


In [15]:
test = pd.read_csv('test_v2.csv')
test

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f25,f26,f27,f28,f29,f30,percent_zeros,percent_ones,population,area
0,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00292,0.00211,...,NaN,NaN,NaN,NaN,0.00498,0.00000,0.976667,0.023333,12635466,2561
1,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00148,0.00176,...,0.91667,0.00000,0.00000,0.00000,0.00000,0.00000,0.976667,0.023333,12635466,2561
2,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00183,0.00183,...,0.59615,0.00427,0.03419,0.00641,0.00498,0.00000,0.976667,0.023333,12635466,2561
3,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00180,0.00210,...,0.66816,0.00897,0.02691,0.00448,0.00459,0.00115,0.976667,0.023333,12635466,2561
4,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00045,0.00078,...,NaN,NaN,NaN,NaN,0.00498,0.00000,0.976667,0.023333,12635466,2561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152761,0,2021-05-01,Республика,Алтай,Турочак,86206915fffffff,52.291052,87.130928,0.00026,0.00025,...,0.82353,0.00000,0.00000,0.70588,0.08824,0.00000,1.000000,0.000000,221559,92903
152762,0,2021-05-01,Республика,Алтай,Чоя,862069577ffffff,51.990728,86.602610,0.00265,0.00226,...,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.000000,0.000000,221559,92903
152763,0,2021-05-01,Республика,Алтай,Улаган,86204dba7ffffff,50.629259,87.948977,0.00057,0.00046,...,1.00000,0.00000,0.14286,0.00000,0.00000,0.00000,1.000000,0.000000,221559,92903
152764,0,2021-05-01,Республика,Алтай,Улита,86206b427ffffff,50.763377,86.161251,0.00034,0.00034,...,NaN,NaN,NaN,NaN,0.00000,0.00000,1.000000,0.000000,221559,92903


In [16]:
test = test.drop(['percent_zeros', 'percent_ones'], axis=1)
test

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f23,f24,f25,f26,f27,f28,f29,f30,population,area
0,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00292,0.00211,...,0.00055,NaN,NaN,NaN,NaN,NaN,0.00498,0.00000,12635466,2561
1,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00148,0.00176,...,0.00000,0.08333,0.91667,0.00000,0.00000,0.00000,0.00000,0.00000,12635466,2561
2,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00183,0.00183,...,0.00055,0.10470,0.59615,0.00427,0.03419,0.00641,0.00498,0.00000,12635466,2561
3,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00180,0.00210,...,0.00000,0.13004,0.66816,0.00897,0.02691,0.00448,0.00459,0.00115,12635466,2561
4,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00045,0.00078,...,0.00055,NaN,NaN,NaN,NaN,NaN,0.00498,0.00000,12635466,2561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152761,0,2021-05-01,Республика,Алтай,Турочак,86206915fffffff,52.291052,87.130928,0.00026,0.00025,...,0.00000,0.00000,0.82353,0.00000,0.00000,0.70588,0.08824,0.00000,221559,92903
152762,0,2021-05-01,Республика,Алтай,Чоя,862069577ffffff,51.990728,86.602610,0.00265,0.00226,...,0.00000,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,221559,92903
152763,0,2021-05-01,Республика,Алтай,Улаган,86204dba7ffffff,50.629259,87.948977,0.00057,0.00046,...,0.00000,0.00000,1.00000,0.00000,0.14286,0.00000,0.00000,0.00000,221559,92903
152764,0,2021-05-01,Республика,Алтай,Улита,86206b427ffffff,50.763377,86.161251,0.00034,0.00034,...,0.00000,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,221559,92903


In [17]:
test_zp_merged = test.merge(zp, how='left', on='subject_name')
test_zp_merged

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f24,f25,f26,f27,f28,f29,f30,population,area,salary
0,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00292,0.00211,...,NaN,NaN,NaN,NaN,NaN,0.00498,0.00000,12635466,2561,100070.0
1,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00148,0.00176,...,0.08333,0.91667,0.00000,0.00000,0.00000,0.00000,0.00000,12635466,2561,100070.0
2,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00183,0.00183,...,0.10470,0.59615,0.00427,0.03419,0.00641,0.00498,0.00000,12635466,2561,100070.0
3,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00180,0.00210,...,0.13004,0.66816,0.00897,0.02691,0.00448,0.00459,0.00115,12635466,2561,100070.0
4,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00045,0.00078,...,NaN,NaN,NaN,NaN,NaN,0.00498,0.00000,12635466,2561,100070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152761,0,2021-05-01,Республика,Алтай,Турочак,86206915fffffff,52.291052,87.130928,0.00026,0.00025,...,0.00000,0.82353,0.00000,0.00000,0.70588,0.08824,0.00000,221559,92903,36269.0
152762,0,2021-05-01,Республика,Алтай,Чоя,862069577ffffff,51.990728,86.602610,0.00265,0.00226,...,0.00000,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,221559,92903,36269.0
152763,0,2021-05-01,Республика,Алтай,Улаган,86204dba7ffffff,50.629259,87.948977,0.00057,0.00046,...,0.00000,1.00000,0.00000,0.14286,0.00000,0.00000,0.00000,221559,92903,36269.0
152764,0,2021-05-01,Республика,Алтай,Улита,86206b427ffffff,50.763377,86.161251,0.00034,0.00034,...,NaN,NaN,NaN,NaN,NaN,0.00000,0.00000,221559,92903,36269.0


In [27]:
test_zp_merged.to_csv('test_v3_no_counts.csv', index=False)

In [18]:
ones = train[train['label']==1].groupby(['hex'])['label'].count()
ones_df = pd.DataFrame(ones).rename({'label':'ones'}, axis=1).reset_index()
zeros = train[train['label']==0].groupby(['hex'])['label'].count()
zeros_df = pd.DataFrame(zeros).rename({'label':'zeros'}, axis=1).reset_index()
df_temp = zeros_df.merge(ones_df, how='outer', on='hex').fillna(0)
df_temp['sum'] = df_temp['zeros'] + df_temp['ones']
df_temp

,hex,zeros,ones,sum
0,86013400fffffff,11.0,7.0,18.0
1,860134047ffffff,22.0,5.0,27.0
2,860134077ffffff,1.0,2.0,3.0
3,860134147ffffff,10.0,4.0,14.0
4,860134167ffffff,4.0,0.0,4.0
...,...,...,...,...
7245,861f735a7ffffff,0.0,1.0,1.0
7246,86206d58fffffff,0.0,1.0,1.0
7247,86214420fffffff,0.0,1.0,1.0
7248,862145717ffffff,0.0,1.0,1.0


In [19]:
ones_zero_train = train_zp_merged.merge(df_temp, how='outer', on='hex')

In [20]:
ones_zero_train

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f29,f30,percent_zeros,percent_ones,population,area,salary,zeros,ones,sum
0,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00101,0.00103,...,0.01737,0.0,0.975096,0.024904,12635466,2561,100070.0,509.0,13.0,522.0
1,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00315,0.00322,...,0.00784,0.0,0.975096,0.024904,12635466,2561,100070.0,509.0,13.0,522.0
2,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00173,0.00175,...,0.01316,0.0,0.975096,0.024904,12635466,2561,100070.0,509.0,13.0,522.0
3,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00000,0.00000,...,0.02703,0.0,0.975096,0.024904,12635466,2561,100070.0,509.0,13.0,522.0
4,1,2020-05-01,Город,Москва,Москва,8611aa7a7ffffff,55.729458,37.516569,0.00035,0.00033,...,0.01737,0.0,0.975096,0.024904,12635466,2561,100070.0,509.0,13.0,522.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294253,0,2020-12-01,Область,Магаданская,Магадан,8617155b7ffffff,59.647301,150.880967,0.00001,0.00001,...,0.03226,0.0,1.000000,0.000000,137767,462464,102843.0,2.0,0.0,2.0
294254,0,2020-12-01,Область,Магаданская,Магадан,86171424fffffff,59.749702,150.900795,NaN,NaN,...,0.03774,0.0,1.000000,0.000000,137767,462464,102843.0,1.0,0.0,1.0
294255,0,2020-12-01,Область,Магаданская,Магадан,861715c87ffffff,59.906078,150.869685,0.00076,0.00086,...,0.08696,0.0,1.000000,0.000000,137767,462464,102843.0,1.0,0.0,1.0
294256,0,2020-12-01,Автономный Округ,Чукотский,Певек,8604ebc5fffffff,69.703205,170.185894,0.00001,0.00002,...,0.00000,0.0,1.000000,0.000000,50040,721481,120641.0,1.0,0.0,1.0


In [26]:
ones_zero_train.to_csv('train_v4.csv', index=False)

In [22]:
ones_zero_train_temp = ones_zero_train[['hex', 'percent_zeros', 'percent_ones',
                                        'zeros', 'ones', 'sum']]
ones_zero_train_temp

,hex,percent_zeros,percent_ones,zeros,ones,sum
0,8611aa7a7ffffff,0.975096,0.024904,509.0,13.0,522.0
1,8611aa7a7ffffff,0.975096,0.024904,509.0,13.0,522.0
2,8611aa7a7ffffff,0.975096,0.024904,509.0,13.0,522.0
3,8611aa7a7ffffff,0.975096,0.024904,509.0,13.0,522.0
4,8611aa7a7ffffff,0.975096,0.024904,509.0,13.0,522.0
...,...,...,...,...,...,...
294253,8617155b7ffffff,1.000000,0.000000,2.0,0.0,2.0
294254,86171424fffffff,1.000000,0.000000,1.0,0.0,1.0
294255,861715c87ffffff,1.000000,0.000000,1.0,0.0,1.0
294256,8604ebc5fffffff,1.000000,0.000000,1.0,0.0,1.0


In [28]:
train_new = pd.read_csv('RTK/train_v6_no_regions.csv')
train_new

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f27,f28,f29,f30,hex-2,count,count0,count1,count0p,count1p
0,1,2020-06-01,Область,Калужская,Козельск,86118461fffffff,54.036155,35.788031,0.00114,0.00137,...,0.0,0.0,0.01152,0.00192,86118461fffffff,100,96,4,0.960000,0.040000
1,1,2020-06-01,Область,Калужская,Курилово,8611819a7ffffff,55.083486,37.013894,0.00010,0.00010,...,0.0,0.0,0.01993,0.00000,8611819a7ffffff,29,26,3,0.896552,0.103448
2,1,2020-06-01,Область,Липецкая,Казинка,86119136fffffff,52.572979,38.545548,0.00006,0.00005,...,0.0,0.0,0.00000,0.00000,86119136fffffff,22,21,1,0.954545,0.045455
3,1,2020-06-01,Область,Орловская,Знаменка,86118640fffffff,52.876245,35.953956,0.00009,0.00005,...,NaN,NaN,0.00000,0.00000,86118640fffffff,29,28,1,0.965517,0.034483
4,1,2020-06-01,Область,Тверская,Крючково,8611a8917ffffff,57.080917,35.559538,0.00025,0.00025,...,0.0,0.0,0.04000,0.00000,8611a8917ffffff,4,3,1,0.750000,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294253,1,2020-06-01,Область,Тверская,Удомля,8611add6fffffff,57.874737,34.991607,0.00106,0.00090,...,0.0,0.0,0.00966,0.00000,8611add6fffffff,123,121,2,0.983740,0.016260
294254,1,2020-06-01,Область,Белгородская,Старый Оскол,861193937ffffff,51.304758,37.888341,0.00079,0.00119,...,0.0,0.0,0.00726,0.00000,861193937ffffff,263,259,4,0.984791,0.015209
294255,1,2020-06-01,Область,Курская,Курск,861194247ffffff,51.747103,36.182261,0.00054,0.00063,...,0.0,0.0,0.00845,0.00000,861194247ffffff,181,179,2,0.988950,0.011050
294256,1,2020-06-01,Область,Рязанская,Рязань,86118e447ffffff,54.602883,39.769765,0.00034,0.00033,...,0.0,0.0,0.02500,0.00000,86118e447ffffff,351,345,6,0.982906,0.017094


In [30]:
train_new_salary = train_new.merge(zp, how='left', on='subject_name')
train_new_salary

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f28,f29,f30,hex-2,count,count0,count1,count0p,count1p,salary
0,1,2020-06-01,Область,Калужская,Козельск,86118461fffffff,54.036155,35.788031,0.00114,0.00137,...,0.0,0.01152,0.00192,86118461fffffff,100,96,4,0.960000,0.040000,43994.0
1,1,2020-06-01,Область,Калужская,Курилово,8611819a7ffffff,55.083486,37.013894,0.00010,0.00010,...,0.0,0.01993,0.00000,8611819a7ffffff,29,26,3,0.896552,0.103448,43994.0
2,1,2020-06-01,Область,Липецкая,Казинка,86119136fffffff,52.572979,38.545548,0.00006,0.00005,...,0.0,0.00000,0.00000,86119136fffffff,22,21,1,0.954545,0.045455,36790.0
3,1,2020-06-01,Область,Орловская,Знаменка,86118640fffffff,52.876245,35.953956,0.00009,0.00005,...,NaN,0.00000,0.00000,86118640fffffff,29,28,1,0.965517,0.034483,31862.0
4,1,2020-06-01,Область,Тверская,Крючково,8611a8917ffffff,57.080917,35.559538,0.00025,0.00025,...,0.0,0.04000,0.00000,8611a8917ffffff,4,3,1,0.750000,0.250000,36077.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294253,1,2020-06-01,Область,Тверская,Удомля,8611add6fffffff,57.874737,34.991607,0.00106,0.00090,...,0.0,0.00966,0.00000,8611add6fffffff,123,121,2,0.983740,0.016260,36077.0
294254,1,2020-06-01,Область,Белгородская,Старый Оскол,861193937ffffff,51.304758,37.888341,0.00079,0.00119,...,0.0,0.00726,0.00000,861193937ffffff,263,259,4,0.984791,0.015209,NaN
294255,1,2020-06-01,Область,Курская,Курск,861194247ffffff,51.747103,36.182261,0.00054,0.00063,...,0.0,0.00845,0.00000,861194247ffffff,181,179,2,0.988950,0.011050,35805.0
294256,1,2020-06-01,Область,Рязанская,Рязань,86118e447ffffff,54.602883,39.769765,0.00034,0.00033,...,0.0,0.02500,0.00000,86118e447ffffff,351,345,6,0.982906,0.017094,36459.0


In [ ]:
subject_name_data = pd.read_csv('subject_name_data.csv')
# delete 'РФ'
subject_name_data = subject_name_data.iloc[:-1, :]

subject_name_data = subject_name_data[(subject_name_data['subject_name']!='Севастополь[a]') &
                  (subject_name_data['subject_name']!='Крым[a]') &
                  (subject_name_data['subject_name']!='Архангельская безНАО ')
                                     & (subject_name_data['subject_name']!='Тюменская областьбез ХМАО и ЯНАО')]

subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('край', '')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('область', '')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('Республика', '')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('без ХМАО и ЯНАО', '')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('(сНАО) ', '')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace(' ', '')

subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('СевернаяОсетия—Алания',
                                                                                  'Северная Осетия - Алания')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('Кемеровская',
                                                                                  'Кемеровская Область-Кузбасс')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('МарийЭл',
                                                                                  'Марий Эл')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('Ямало-Ненецкийавтономныйокруг',
                                                                                  'Ямало-Ненецкий')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('Ненецкийавтономныйокруг',
                                                                                  'Ненецкий')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('Чукотскийавтономныйокруг',
                                                                                  'Чукотский')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('Еврейскаяавтономная',
                                                                                  'Еврейская')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('Еврейскаяавтономная',
                                                                                  'Еврейская')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('Ханты-Мансийскийавтономныйокруг—Югра',
                                                                                  'Ханты-Мансийский Автономный Округ - Югра')

In [ ]:
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('(', '')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace(')', '')

In [ ]:
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('АрхангельскаясНАО',
                                                                                  'Архангельская')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('ТюменскаясХМАОиЯНАО',
                                                                                  'Тюменская')
subject_name_data['subject_name'] = subject_name_data['subject_name'].str.replace('Саха(Якутия)',
                                                                                  'Саха /Якутия/')

subject_name_data = subject_name_data[subject_name_data['subject_name']!='АрхангельскаябезНАО']
subject_name_data = subject_name_data[subject_name_data['subject_name']!='Крым[a]']

In [35]:
subject_name_merged = train_new_salary.merge(subject_name_data, how='inner', on='subject_name')
subject_name_merged

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f30,hex-2,count,count0,count1,count0p,count1p,salary,population,area
0,1,2020-06-01,Область,Калужская,Козельск,86118461fffffff,54.036155,35.788031,0.00114,0.00137,...,0.00192,86118461fffffff,100,96,4,0.960000,0.040000,43994.0,1012844,29777
1,1,2020-06-01,Область,Калужская,Курилово,8611819a7ffffff,55.083486,37.013894,0.00010,0.00010,...,0.00000,8611819a7ffffff,29,26,3,0.896552,0.103448,43994.0,1012844,29777
2,1,2020-06-01,Область,Калужская,Балабаново,8611852b7ffffff,55.178959,36.674532,0.00021,0.00029,...,0.00000,8611852b7ffffff,85,81,4,0.952941,0.047059,43994.0,1012844,29777
3,1,2020-06-01,Область,Калужская,Сосенский,8611846c7ffffff,54.047013,35.944630,0.00193,0.00175,...,0.00000,8611846c7ffffff,87,83,4,0.954023,0.045977,43994.0,1012844,29777
4,1,2020-06-01,Область,Калужская,Обнинск,861185747ffffff,55.086586,36.604952,0.00015,0.00022,...,0.00000,861185747ffffff,253,251,2,0.992095,0.007905,43994.0,1012844,29777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294253,0,2020-12-01,Область,Магаданская,Магадан,8617142dfffffff,59.550070,150.740527,0.00009,0.00005,...,0.00346,8617142dfffffff,72,72,0,1.000000,0.000000,102843.0,137767,462464
294254,0,2020-12-01,Область,Магаданская,Ола,8617154afffffff,59.577462,151.294079,0.00045,0.00046,...,0.00000,8617154afffffff,13,13,0,1.000000,0.000000,102843.0,137767,462464
294255,0,2020-12-01,Область,Магаданская,Магадан,8617142cfffffff,59.598694,150.810630,NaN,NaN,...,0.00000,8617142cfffffff,82,80,2,0.975610,0.024390,102843.0,137767,462464
294256,0,2020-12-01,Автономный Округ,Чукотский,Певек,8604ebc5fffffff,69.703205,170.185894,0.00001,0.00002,...,0.00000,8604ebc5fffffff,1,1,0,1.000000,0.000000,120641.0,50040,721481


In [36]:
subject_name_merged['density'] = subject_name_merged['population'] / subject_name_merged['area']
subject_name_merged

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,hex-2,count,count0,count1,count0p,count1p,salary,population,area,density
0,1,2020-06-01,Область,Калужская,Козельск,86118461fffffff,54.036155,35.788031,0.00114,0.00137,...,86118461fffffff,100,96,4,0.960000,0.040000,43994.0,1012844,29777,34.014306
1,1,2020-06-01,Область,Калужская,Курилово,8611819a7ffffff,55.083486,37.013894,0.00010,0.00010,...,8611819a7ffffff,29,26,3,0.896552,0.103448,43994.0,1012844,29777,34.014306
2,1,2020-06-01,Область,Калужская,Балабаново,8611852b7ffffff,55.178959,36.674532,0.00021,0.00029,...,8611852b7ffffff,85,81,4,0.952941,0.047059,43994.0,1012844,29777,34.014306
3,1,2020-06-01,Область,Калужская,Сосенский,8611846c7ffffff,54.047013,35.944630,0.00193,0.00175,...,8611846c7ffffff,87,83,4,0.954023,0.045977,43994.0,1012844,29777,34.014306
4,1,2020-06-01,Область,Калужская,Обнинск,861185747ffffff,55.086586,36.604952,0.00015,0.00022,...,861185747ffffff,253,251,2,0.992095,0.007905,43994.0,1012844,29777,34.014306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294253,0,2020-12-01,Область,Магаданская,Магадан,8617142dfffffff,59.550070,150.740527,0.00009,0.00005,...,8617142dfffffff,72,72,0,1.000000,0.000000,102843.0,137767,462464,0.297898
294254,0,2020-12-01,Область,Магаданская,Ола,8617154afffffff,59.577462,151.294079,0.00045,0.00046,...,8617154afffffff,13,13,0,1.000000,0.000000,102843.0,137767,462464,0.297898
294255,0,2020-12-01,Область,Магаданская,Магадан,8617142cfffffff,59.598694,150.810630,NaN,NaN,...,8617142cfffffff,82,80,2,0.975610,0.024390,102843.0,137767,462464,0.297898
294256,0,2020-12-01,Автономный Округ,Чукотский,Певек,8604ebc5fffffff,69.703205,170.185894,0.00001,0.00002,...,8604ebc5fffffff,1,1,0,1.000000,0.000000,120641.0,50040,721481,0.069357


In [37]:
# subject_name_merged.to_csv('train_v7.csv', index=False)

In [42]:
test = pd.read_csv('test_v6_no_regions.csv')
test

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f27,f28,f29,f30,hex-2,count,count0,count1,count0p,count1p
0,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00292,0.00211,...,NaN,NaN,0.00498,0.00000,8611aa6b7ffffff,562.0,545.0,17.0,0.969751,0.030249
1,1,2020-11-01,Город,Москва,Москва,8611aa70fffffff,55.879910,37.583383,0.00265,0.00355,...,NaN,NaN,0.00333,0.00000,8611aa70fffffff,1127.0,1091.0,36.0,0.968057,0.031943
2,1,2020-11-01,Город,Москва,Москва,8611aa627ffffff,55.855726,37.669858,0.00046,0.00059,...,NaN,NaN,0.00904,0.00000,8611aa627ffffff,721.0,698.0,23.0,0.968100,0.031900
3,1,2020-11-01,Город,Москва,Москва,861181b2fffffff,55.496542,37.542584,0.00304,0.00243,...,0.00993,0.00000,0.00977,0.00122,861181b2fffffff,99.0,93.0,6.0,0.939394,0.060606
4,1,2020-11-01,Город,Москва,Москва,8611aa4cfffffff,55.613025,37.529612,0.00183,0.00163,...,NaN,NaN,0.00617,0.00000,8611aa4cfffffff,689.0,668.0,21.0,0.969521,0.030479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152761,0,2021-06-01,Область,Магаданская,Магадан,861710967ffffff,59.544997,150.861279,0.00164,0.00287,...,0.00296,0.05769,0.05550,0.00218,861710967ffffff,176.0,173.0,3.0,0.982955,0.017045
152762,0,2021-06-01,Край,Камчатский,Петропавловск-Камчатский,8616163afffffff,53.090818,158.588644,0.00053,0.00063,...,0.00272,0.08832,0.08333,0.00252,8616163afffffff,146.0,143.0,3.0,0.979452,0.020548
152763,0,2021-06-01,Край,Приморский,Покровка,863161b07ffffff,43.941627,131.648920,NaN,NaN,...,0.01449,0.02899,NaN,NaN,863161b07ffffff,43.0,42.0,1.0,0.976744,0.023256
152764,0,2021-06-01,Республика,Саха /Якутия/,Якутск,86155d14fffffff,62.007715,129.645173,0.00058,0.00068,...,NaN,NaN,0.06130,0.00060,86155d14fffffff,409.0,406.0,3.0,0.992665,0.007335


In [43]:
test_new_salary = test.merge(zp, how='left', on='subject_name')
test_new_salary

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f28,f29,f30,hex-2,count,count0,count1,count0p,count1p,salary
0,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00292,0.00211,...,NaN,0.00498,0.00000,8611aa6b7ffffff,562.0,545.0,17.0,0.969751,0.030249,100070.0
1,1,2020-11-01,Город,Москва,Москва,8611aa70fffffff,55.879910,37.583383,0.00265,0.00355,...,NaN,0.00333,0.00000,8611aa70fffffff,1127.0,1091.0,36.0,0.968057,0.031943,100070.0
2,1,2020-11-01,Город,Москва,Москва,8611aa627ffffff,55.855726,37.669858,0.00046,0.00059,...,NaN,0.00904,0.00000,8611aa627ffffff,721.0,698.0,23.0,0.968100,0.031900,100070.0
3,1,2020-11-01,Город,Москва,Москва,861181b2fffffff,55.496542,37.542584,0.00304,0.00243,...,0.00000,0.00977,0.00122,861181b2fffffff,99.0,93.0,6.0,0.939394,0.060606,100070.0
4,1,2020-11-01,Город,Москва,Москва,8611aa4cfffffff,55.613025,37.529612,0.00183,0.00163,...,NaN,0.00617,0.00000,8611aa4cfffffff,689.0,668.0,21.0,0.969521,0.030479,100070.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152761,0,2021-06-01,Область,Магаданская,Магадан,861710967ffffff,59.544997,150.861279,0.00164,0.00287,...,0.05769,0.05550,0.00218,861710967ffffff,176.0,173.0,3.0,0.982955,0.017045,102843.0
152762,0,2021-06-01,Край,Камчатский,Петропавловск-Камчатский,8616163afffffff,53.090818,158.588644,0.00053,0.00063,...,0.08832,0.08333,0.00252,8616163afffffff,146.0,143.0,3.0,0.979452,0.020548,85623.0
152763,0,2021-06-01,Край,Приморский,Покровка,863161b07ffffff,43.941627,131.648920,NaN,NaN,...,0.02899,NaN,NaN,863161b07ffffff,43.0,42.0,1.0,0.976744,0.023256,50105.0
152764,0,2021-06-01,Республика,Саха /Якутия/,Якутск,86155d14fffffff,62.007715,129.645173,0.00058,0.00068,...,NaN,0.06130,0.00060,86155d14fffffff,409.0,406.0,3.0,0.992665,0.007335,77178.0


In [44]:
subject_name_merged = test_new_salary.merge(subject_name_data, how='inner', on='subject_name')
subject_name_merged

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,f30,hex-2,count,count0,count1,count0p,count1p,salary,population,area
0,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00292,0.00211,...,0.00000,8611aa6b7ffffff,562.0,545.0,17.0,0.969751,0.030249,100070.0,12635466,2561
1,1,2020-11-01,Город,Москва,Москва,8611aa70fffffff,55.879910,37.583383,0.00265,0.00355,...,0.00000,8611aa70fffffff,1127.0,1091.0,36.0,0.968057,0.031943,100070.0,12635466,2561
2,1,2020-11-01,Город,Москва,Москва,8611aa627ffffff,55.855726,37.669858,0.00046,0.00059,...,0.00000,8611aa627ffffff,721.0,698.0,23.0,0.968100,0.031900,100070.0,12635466,2561
3,1,2020-11-01,Город,Москва,Москва,861181b2fffffff,55.496542,37.542584,0.00304,0.00243,...,0.00122,861181b2fffffff,99.0,93.0,6.0,0.939394,0.060606,100070.0,12635466,2561
4,1,2020-11-01,Город,Москва,Москва,8611aa4cfffffff,55.613025,37.529612,0.00183,0.00163,...,0.00000,8611aa4cfffffff,689.0,668.0,21.0,0.969521,0.030479,100070.0,12635466,2561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152761,0,2021-05-01,Республика,Алтай,Владимировка,86206a997ffffff,51.036626,84.209675,0.00021,0.00019,...,0.00000,86206a997ffffff,2.0,2.0,0.0,1.000000,0.000000,36269.0,221559,92903
152762,0,2021-05-01,Республика,Алтай,Горно-Алтайск,862068357ffffff,51.962374,85.949424,0.00096,0.00074,...,0.00000,862068357ffffff,123.0,121.0,2.0,0.983740,0.016260,36269.0,221559,92903
152763,0,2021-05-01,Республика,Алтай,Майма,862068377ffffff,52.018114,85.903139,0.00348,0.00357,...,0.00000,862068377ffffff,36.0,35.0,1.0,0.972222,0.027778,36269.0,221559,92903
152764,0,2021-05-01,Республика,Алтай,Горно-Алтайск,862068357ffffff,51.962374,85.949424,0.00006,0.00006,...,0.00000,862068357ffffff,123.0,121.0,2.0,0.983740,0.016260,36269.0,221559,92903


In [45]:
subject_name_merged['density'] = subject_name_merged['population'] / subject_name_merged['area']
subject_name_merged

,label,period,subject_type,subject_name,city_name,hex,hex_lat,hex_lon,f1,f2,...,hex-2,count,count0,count1,count0p,count1p,salary,population,area,density
0,1,2020-11-01,Город,Москва,Москва,8611aa6b7ffffff,55.656639,37.774902,0.00292,0.00211,...,8611aa6b7ffffff,562.0,545.0,17.0,0.969751,0.030249,100070.0,12635466,2561,4933.801640
1,1,2020-11-01,Город,Москва,Москва,8611aa70fffffff,55.879910,37.583383,0.00265,0.00355,...,8611aa70fffffff,1127.0,1091.0,36.0,0.968057,0.031943,100070.0,12635466,2561,4933.801640
2,1,2020-11-01,Город,Москва,Москва,8611aa627ffffff,55.855726,37.669858,0.00046,0.00059,...,8611aa627ffffff,721.0,698.0,23.0,0.968100,0.031900,100070.0,12635466,2561,4933.801640
3,1,2020-11-01,Город,Москва,Москва,861181b2fffffff,55.496542,37.542584,0.00304,0.00243,...,861181b2fffffff,99.0,93.0,6.0,0.939394,0.060606,100070.0,12635466,2561,4933.801640
4,1,2020-11-01,Город,Москва,Москва,8611aa4cfffffff,55.613025,37.529612,0.00183,0.00163,...,8611aa4cfffffff,689.0,668.0,21.0,0.969521,0.030479,100070.0,12635466,2561,4933.801640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152761,0,2021-05-01,Республика,Алтай,Владимировка,86206a997ffffff,51.036626,84.209675,0.00021,0.00019,...,86206a997ffffff,2.0,2.0,0.0,1.000000,0.000000,36269.0,221559,92903,2.384842
152762,0,2021-05-01,Республика,Алтай,Горно-Алтайск,862068357ffffff,51.962374,85.949424,0.00096,0.00074,...,862068357ffffff,123.0,121.0,2.0,0.983740,0.016260,36269.0,221559,92903,2.384842
152763,0,2021-05-01,Республика,Алтай,Майма,862068377ffffff,52.018114,85.903139,0.00348,0.00357,...,862068377ffffff,36.0,35.0,1.0,0.972222,0.027778,36269.0,221559,92903,2.384842
152764,0,2021-05-01,Республика,Алтай,Горно-Алтайск,862068357ffffff,51.962374,85.949424,0.00006,0.00006,...,862068357ffffff,123.0,121.0,2.0,0.983740,0.016260,36269.0,221559,92903,2.384842


In [48]:
subject_name_merged[['label', 'subject_name', 'count', 'count0', 'count1',
                     'count0p', 'count1p', 'salary', 'population', 'area', 'density']].head()

,label,subject_name,count,count0,count1,count0p,count1p,salary,population,area,density
0,1,Москва,562.0,545.0,17.0,0.969751,0.030249,100070.0,12635466,2561,4933.80164
1,1,Москва,1127.0,1091.0,36.0,0.968057,0.031943,100070.0,12635466,2561,4933.80164
2,1,Москва,721.0,698.0,23.0,0.968100,0.031900,100070.0,12635466,2561,4933.80164
3,1,Москва,99.0,93.0,6.0,0.939394,0.060606,100070.0,12635466,2561,4933.80164
4,1,Москва,689.0,668.0,21.0,0.969521,0.030479,100070.0,12635466,2561,4933.80164


In [49]:
subject_name_merged.columns

Index(['label', 'period', 'subject_type', 'subject_name', 'city_name', 'hex',
       'hex_lat', 'hex_lon', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8',
       'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18',
       'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28',
       'f29', 'f30', 'hex-2', 'count', 'count0', 'count1', 'count0p',
       'count1p', 'salary', 'population', 'area', 'density'],
      dtype='object')

In [46]:
# subject_name_merged.to_csv('test_v7.csv', index=False)